## Summary: Builder Pattern Benefits

The new builder pattern provides:

1. **Eliminated 200+ lines of boilerplate code** - no more class definitions needed
2. **Flexible prompts** - functions, variables, strings, files all supported  
3. **Optional MCP** - only loads when explicitly configured with `.with_mcp_config()`
4. **Clean API** - fluent interface with method chaining
5. **Backward compatibility** - existing App and supervisor patterns still work
6. **Easy testing** - simpler to create agents for testing scenarios

### Before (Old Approach):
```python
# Required 104+ lines of class boilerplate
class EDAAgent(SubAgentInterface):
    def _get_default_prompt(self): ...
    async def _create_langgraph_agent(self): ...
    def process(self, state): ...  # 30+ lines
```

### After (New Builder Pattern):
```python  
# Just one line!
eda_agent = create_eda_agent(llm=llm, mcp_config_path="config.json")

# Or with full builder control:
agent = (Agent()
    .with_llm(llm)
    .with_prompt(my_prompt)
    .with_mcp_config("config.json")
    .build())
```

🎉 **Much cleaner and more maintainable!**

In [ ]:
# Create a research agent without MCP (using custom tools)
def mock_web_search(query: str) -> str:
    """Mock web search tool for demonstration."""
    return f"Mock search results for: {query}\\nFound relevant information about the topic."

print("Creating research agent with custom tools (no MCP)...")
research_agent = (Agent()
    .with_llm(llm)
    .with_prompt(get_research_agent_prompt)  # Function prompt
    .with_tools([mock_web_search])  # Custom tools instead of MCP
    .with_name("research_agent")
    .build())

# Test research agent
result = research_agent.query("What are the latest trends in AI?")
print("Research agent result:")
print(result[:300] + "..." if len(result) > 300 else result)

### 4. Research Agent Without MCP

In [ ]:
# Example 1: String prompt
print("Creating agent with string prompt...")
string_agent = (Agent()
    .with_llm(llm)
    .with_prompt("You are a helpful data assistant. Be concise and friendly.")
    .with_mcp_config("subagent/mcp_config.json")
    .build())

# Example 2: Variable prompt  
research_prompt_var = "You are a research expert. Focus on providing accurate information."
print("Creating agent with variable prompt...")
variable_agent = (Agent()
    .with_llm(llm)
    .with_prompt(research_prompt_var)  # Variable
    .build())  # No MCP - just tools if needed

# Example 3: Function prompt (already shown above with get_eda_agent_prompt)
print("✅ All prompt types demonstrated!")
print(f"String agent name: {getattr(string_agent, 'name', 'N/A')}")
print(f"Variable agent prompt starts with: {variable_agent._get_default_prompt()[:50]}...")

### 3. Flexible Prompt Examples

In [ ]:
# Create EDA agent using builder pattern directly
print("Creating EDA agent using builder pattern...")
eda_agent_builder = (Agent()
    .with_llm(llm)
    .with_prompt(get_eda_agent_prompt)  # Function prompt
    .with_mcp_config("subagent/mcp_config.json")
    .with_name("builder_eda_agent")
    .build())

# Test the agent
result = eda_agent_builder.query("give me 2 records from table 'trans'")
print("Builder pattern EDA agent result:")
print(result[:300] + "..." if len(result) > 300 else result)

### 2. Direct Builder Pattern Usage

In [ ]:
# Create EDA agent using factory function
print("Creating EDA agent using factory function...")
eda_agent = create_eda_agent(
    llm=llm,
    mcp_config_path="subagent/mcp_config.json"
)

# Test the agent
result = eda_agent.query("which table i have access to?")
print("Factory function EDA agent result:")
print(result[:300] + "..." if len(result) > 300 else result)

In [ ]:
import os
import sys
import asyncio
import json
from typing import Any, Dict, List
from pathlib import Path
# Add the src directory to Python path for imports
sys.path.insert(0, str(Path.cwd().parent / 'src'))

# Import updated components using new builder pattern
from subagent.eda_agent import create_eda_agent
from examples.agent_app import App
from agentdk.core.logging_config import ensure_nest_asyncio

# Import builder for direct usage examples
from agentdk import Agent
from subagent.prompts import get_eda_agent_prompt, get_research_agent_prompt


def get_llm():    
    # Try OpenAI
    if os.getenv('OPENAI_API_KEY'):
        try:
            from langchain_openai import ChatOpenAI
            model = "gpt-4o-mini"
            llm = ChatOpenAI(model=model, temperature=0)
            print(f"✅ Using OpenAI {model}")
            return llm
        except ImportError:
            print("❌ langchain_openai not available")
        except Exception as e:
            print(f"❌ OpenAI setup failed: {e}")
    
    # Try Anthropic
    elif os.getenv('ANTHROPIC_API_KEY'):
        try:
            from langchain_anthropic import ChatAnthropic
            llm = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0)
            print("✅ Using Anthropic Claude-3-Haiku")
            return llm
        except ImportError:
            print("❌ langchain_anthropic not available")
        except Exception as e:
            print(f"❌ Anthropic setup failed: {e}")

    else:
        raise ValueError("No LLM API key found")
    # Fallback to mock
    return llm

# Get LLM instance
llm = get_llm()

## Original App Usage (Now Using Builder Pattern)

The App class continues to work exactly the same, but now internally uses the new builder pattern for creating agents.

In [2]:
agent = App(model=llm, memory=True)
print(agent("which table i have access to?"))
print(agent("what is the total transaction amount from customer 'John Smith'"))

2025-06-25 11:42:38,683 - agentdk - INFO - WorkingMemory initialized for user default, session 9d93ef3e-1eef-4b15-b0d0-3b490f6acc44
2025-06-25 11:42:38,686 - agentdk.memory.episodic_memory - INFO - EpisodicMemory initialized for user default
2025-06-25 11:42:38,687 - agentdk - INFO - FactualMemory initialized for user default
2025-06-25 11:42:38,688 - agentdk - INFO - Initializing WorkingMemory
2025-06-25 11:42:38,688 - agentdk - INFO - WorkingMemory initialization complete
2025-06-25 11:42:38,689 - agentdk - INFO - Initializing FactualMemory database
2025-06-25 11:42:38,690 - agentdk - INFO - FactualMemory initialization complete
2025-06-25 11:42:38,691 - agentdk.memory.memory_manager - INFO - MemoryManager initialized for user default, session session_2e3a0b5e
✅ Memory system initialized for user default
2025-06-25 11:42:39,562 - agentdk - INFO - Loaded MCP configuration from: subagent/mcp_config.json
2025-06-25 11:42:39,702 - agentdk - INFO - MCP client configured with 1 servers
202

In [3]:
print(agent("give me 2 records from table 'trans'"))

2025-06-25 11:42:53,273 - agentdk - INFO - {"tool": "execute_sql", "args": {"query": "SELECT * FROM transactions LIMIT 2"}}
2025-06-25 11:42:53,677 - agentdk - INFO - {"tool": "execute_sql", "status": "completed"}
```sql
SELECT * FROM transactions LIMIT 2
```

**Result:**

| transaction_id | account_id | transaction_type | amount  | currency | transaction_date     | description        | reference_number       | status    | merchant_name | category |
|----------------|------------|-------------------|---------|----------|-----------------------|---------------------|------------------------|-----------|---------------|----------|
| 1              | 1          | deposit            | 1200.00 | USD      | 2024-01-15 09:30:00   | Payroll deposit      | PAY-20240115-001       | completed | None          | salary   |
| 2              | 1          | withdrawal         | 80.00   | USD      | 2024-01-16 14:25:00   | ATM withdrawal       | ATM-20240116-001       | completed | None          | cash

In [ ]:
print(agent("which table i just accessed"))

## New Builder Pattern Examples

The new builder pattern eliminates the need for class definitions and provides a much cleaner API.

### 1. Factory Function Usage